In [1]:
#El porcentaje de contenidos que están publicados cuya última edición no tiene comentario para los usuarios que realizaron 1, >10 y >100 de las últimas ediciones (⭐⭐)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [5]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [6]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/Organizacion De Datos/Trabajo_practico/contents.csv', header=True, inferSchema=True)
rdd = df.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


**DISCLAIMER: Lo dejo todo particionado con take´s para que se lea mejor.**

No tomo en cuenta aquellos usuarios que no tengan id (que no esten registrados, ya que no deberían poder editar)

In [15]:
rdd_filtrado = rdd.filter(lambda x:x.revisor_id!=None)

obtengo de la forma (revisor_id, (#ediciones sin coment, #ediciones totales)

In [16]:
rdd_filtrado.map(lambda x: (x.revisor_id , (x.revisor_comment==None, 1)) ).reduceByKey(lambda a,b:(a[0]+b[0],a[1]+b[1])).take(5)

[('3615366', (238, 2905)),
 ('6331749', (38, 47)),
 ('15514', (44, 1216)),
 ('1237851', (104, 760)),
 ('6179998', (False, 1))]

funciones para setear los grupos en el flatmap que uso luego

In [9]:
def elegir_grupo_1(x):
  if x[1][1] == 1:
    return (x[1][0],x[1][1])
  
  else:
    return (0,0)

def elegir_grupo_10(x):
  if x[1][1] > 10:
    return (x[1][0],x[1][1])
  
  else:
    return (0,0)

def elegir_grupo_100(x):
  if x[1][1] > 100:
    return (x[1][0],x[1][1])
  
  else:
    return (0,0)

por cada registro, obtengo 3 que me dicen si pertenece al grupo indicado en la clave (revisiones sin coment, revisiones totales) o si no (0,0)

In [20]:
rdd_filtrado.map(lambda x: (x.revisor_id , (x.revisor_comment==None, 1)) ).reduceByKey(lambda a,b:(a[0]+b[0],a[1]+b[1])).flatMap(
    lambda x:[ ('1',(elegir_grupo_1(x))),('10',(elegir_grupo_10(x)) ), ('100',(elegir_grupo_100(x))) ]).take(6)

[('1', (0, 0)),
 ('10', (238, 2905)),
 ('100', (238, 2905)),
 ('1', (0, 0)),
 ('10', (38, 47)),
 ('100', (0, 0))]

sumo por clave y por valor 

In [21]:
rdd_filtrado.map(lambda x: (x.revisor_id , (x.revisor_comment==None, 1)) ).reduceByKey(lambda a,b:(a[0]+b[0],a[1]+b[1])).flatMap(
    lambda x:[ ('1',(elegir_grupo_1(x))),('10',(elegir_grupo_10(x)) ), ('100',(elegir_grupo_100(x))) ]).reduceByKey(lambda a,b:(a[0]+b[0], a[1]+b[1])).take(6)

[('10', (373713, 3862375)), ('100', (343424, 3719443)), ('1', (6349, 24776))]

obtengo porcentajes

In [19]:
rdd_filtrado.map(lambda x: (x.revisor_id , (x.revisor_comment==None, 1)) ).reduceByKey(lambda a,b:(a[0]+b[0],a[1]+b[1])).flatMap(
    lambda x:[ ('1',(elegir_grupo_1(x))),('10',(elegir_grupo_10(x)) ), ('100',(elegir_grupo_100(x))) ]).reduceByKey(lambda a,b:(a[0]+b[0], a[1]+b[1])).map(lambda x:(x[0],x[1][0]*100/x[1][1])).take(3)

[('10', 9.675730606168484),
 ('100', 9.233210456511902),
 ('1', 25.625605424604455)]